# Recreating Feels Like temperatures with an ANN

In [ ]:
%matplotlib inline

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd
import copy

Let's load in our ASOS station dataset.

In [ ]:
input_data = pd.read_csv("../Week_5/al_asos_jul_23_v2.csv")

In [ ]:
input_data

In [ ]:
# It is often good practice to drop missing data, so let's do just that. 
input_data_nona = input_data.dropna()

In [ ]:
input_data_nona

583 rows!?! That's not enough. Remember to be careful as you're doing this...

In [ ]:
input_data_nona = input_data.drop('gust', axis=1).dropna()

In [ ]:
input_data_nona

See any issues here?

In [ ]:
input_data_renamed_clouds = copy.deepcopy(input_data)
all_cloud_vals = input_data_renamed_clouds['skyc1']
all_cloud_vals.fillna('CLR')
input_data_renamed_clouds['skyc1'] = all_cloud_vals
input_data_renamed_clouds

In [ ]:
input_data_nona = input_data_renamed_clouds.drop(['gust', 'skyl1', 'mslp'], axis=1).dropna()

In [ ]:
input_data_nona

In [ ]:
plt.scatter(input_data_nona['tmpf'], input_data_nona['feel'])
plt.xlabel("Temperature (F)")
plt.ylabel("Heat index (F)")

In [ ]:
def nws_heat_index(T, RH):
    '''
    T in fahrenheit, RH in %
    https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml
    '''
    initial_hi =  0.5 * (T + 61.0 + ((T-68.0)*1.2) + (RH*0.094))
    if initial_hi < 80:
        return initial_hi

    adjustment_factor = 0
    if T>=80 and T<=112 and RH <13:
        adjustment_factor =  ((13-RH)/4)*np.sqrt((17-np.abs(T-95.))/17)
    
    if T>=80 and T<=87 and RH > 85:
         adjustment_factor = ((RH-85)/10) * ((87-T)/5)

    return (-42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - 
           .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH) + adjustment_factor

nws_heat_index_vectorized = np.vectorize(nws_heat_index)

In [ ]:
our_calc_hi = nws_heat_index_vectorized(input_data_nona['tmpf'], input_data_nona['relh'])

In [ ]:
plt.scatter(our_calc_hi, input_data_nona['feel'])
plt.xlabel("Found HI Equation (F)")
plt.ylabel("IEM Reported Feels Like")

In [ ]:
plt.scatter(our_calc_hi, input_data_nona['feel'])
plt.xlabel("Found HI Equation (F)")
plt.ylabel("IEM Reported Feels Like")
plt.xlim(80, 120)
plt.ylim(80, 120)

Let's try this with an ANN

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Prepare our data first
training_data, test_validation = train_test_split(input_data_nona, test_size=0.25)
testing_data, validation_data = train_test_split(test_validation, test_size=0.5)

In [ ]:
training_data['tmpf'].values.shape

In [ ]:
np.sum(pd.isna(training_data['tmpf'].values))

In [ ]:
# Let's try adding different layers
input1 = keras.layers.Input(shape=(2, ))
input_norm = keras.layers.Normalization()(input1)


dense1 = keras.layers.Dense(128, input_dim=0, activation=keras.activations.sigmoid)(input_norm)
dense2 = keras.layers.Dense(64, input_dim=0, activation=keras.activations.relu)(dense1)

output = keras.layers.Dense(1, activation=keras.activations.relu, use_bias=True)(dense2)
model = keras.models.Model(inputs=[input1], outputs=[output])
model.compile(optimizer=keras.optimizers.legacy.Adam(learning_rate=0.01), loss='mean_squared_error')
model.summary()

In [ ]:
model.fit(training_data[['tmpf', 'relh']].values, 
           training_data['feel'].values, epochs=3, batch_size=64)

In [ ]:
predicted_hi = model.predict(testing_data[['tmpf', 'relh']].values)

In [ ]:
plt.scatter(predicted_hi, testing_data['feel'])
plt.xlabel("Predicted from model (F)")
plt.ylabel("IEM Reported Feels Like")
#plt.xlim(50, 120)
#plt.ylim(50, 120)
plt.plot([50, 120], [50, 120], color='grey', ls='--')

In [ ]:
from sklearn.inspection import PartialDependenceDisplay


In [ ]:
model.dummy_ = "dummy"
model._estimator_type = "regressor"

In [ ]:
my_plots = PartialDependenceDisplay.from_estimator(model,       
                                   features=[0, 1], # column numbers of plots we want to show
                                   X=training_data[['tmpf', 'relh']].values,            # raw predictors data.
                                   feature_names=['Temperature (F)', 'Relative Humidity (%)'])
